In [49]:
import pandas as pd
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer

df=pd.read_csv('C:\\temp\\2020_July_items_master.csv')

Part_Desc='PART_DESC'

df['PART NO_DESC']=df[Part_Desc].str.strip()
# df['PART NO_DESC']=df[Part_Desc].str.replace('-',",", regex=True)
# df['PART NO_DESC']=df[Part_Desc].str.upper()

df.rename(columns={'Items_': 'items_'},inplace=True)


In [28]:
from sklearn.model_selection import train_test_split
desc=df['PART NO_DESC'].values
item=df['items_'].values
Xd_train, Xd_test, y_train, y_test= train_test_split(desc, item, test_size=0.3)


In [29]:
vectorizer=CountVectorizer(min_df=0.0001,   strip_accents='unicode',)
X_train=vectorizer.fit_transform(Xd_train)
X_test=vectorizer.transform(Xd_test)
X_train

<803x1473 sparse matrix of type '<class 'numpy.int64'>'
	with 5406 stored elements in Compressed Sparse Row format>

# Base model

In [36]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
score

0.9536231884057971

In [37]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.936231884057971


In [73]:
from sklearn.svm import SVC
# this_C=1
# this_gamma=0.01
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
svc=SVC()
clf2=GridSearchCV(svc,param_grid, refit=True, verbose=1)
clf2.fit(X_train, y_train)


print (grid.best_params_)
# print(grid.best_estimator_)

grid_predictions = clf2.predict(X_test)
# print classification report
print(classification_report(y_test, grid_predictions))
      

# clf2=SVC(kernel='linear',C=this_C, gamma=this_gamma).fit(X_train, y_train)
# score=clf2.score(X_test, y_test)

# print (score)

# # print (y_test)
predict=clf2.predict(X_test)

t_result=pd.DataFrame({'y_test':y_test, 'predict':predict})
t_result.to_csv('c:\\temp\\t_result2.csv')

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits
{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
                                  precision    recall  f1-score   support

                     ACC Emitter       1.00      1.00      1.00         4
                Apperture Driver       0.00      0.00      0.00         2
                          Column       1.00      1.00      1.00         3
                        Computer       0.92      1.00      0.96        22
                   Contact Tower       1.00      1.00      1.00         1
                             DIC       1.00      1.00      1.00         8
                             DWG       1.00      1.00      1.00         8
               Deflection Driver       1.00      1.00      1.00         7
                         E CHUCK       1.00      1.00      1.00         1
         E-CHUCK/GROUNDER DRIVER       1.00      1.00      1.00         6
                             Gun       0.71      1.00      0.83         5
        

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   12.4s finished
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# from sklearn.tree import DecisionTreeClassifier
import pydotplus
clf=DecisionTreeClassifier()
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)


# predict all data to trace back the in accuracy

In [7]:
Xd_all=desc
y_all=item
X_all=vectorizer.transform(Xd_all)
score=clf2.score(X_all, y_all)
print (score)
predict=clf2.predict(X_all)
df['predict']=predict
df.to_csv('c:\\temp\\all_result.csv')


0.9843205574912892


## Apply to New Dataset

In [10]:
dft=pd.read_csv('c:\\temp\\blah-testq4.csv')
dft

print (dft.columns)
len(dft['PART_DESC'])

###########**** make sure no empty end-row  *****########## 

Index(['Source Table', 'MACHINE_REGION', 'REF_NO', 'CUSTOMER', 'MACHINE_NO',
       'PARTS_REQUEST_REASON_CODE', 'APPLY_DATE', 'NEED_BY', 'SO_NO',
       'SO_LINE', 'PART_NO', 'PART_DESC', 'NEED_TO_BE_RETURNED', 'SHIPPED_QTY',
       'SHIPPING_ORG', 'SUBINVENTORY', 'DELIVERY_NUM', 'ACTUAL_SHIPPED_DATE',
       'APPROVED_DATE_2NDLINE', 'PRIORITY', 'FILLER', 'APPLICANT',
       'PARTS_REQUEST_STATUS', 'INV_AVAILABLE_DATE', 'INSTALLED_DATE',
       'NON_INSTALLED_REASON', 'NON_INSTALLED_RETURN', 'NON_INSTALL_DAYS',
       'INSTALLATION_STATUS', 'RMA_NO', 'RETURN_DATE', 'FORM_STATUS',
       'RETURNED_QTY', 'HMI_SN', 'Items_'],
      dtype='object')


292

In [9]:
Part_Desc='PART_DESC'
dft['PART NO_DESC']=dft[Part_Desc].str.strip()
dft['PART NO_DESC']=dft[Part_Desc].str.replace('-',",", regex=True)
dft['PART NO_DESC']=dft[Part_Desc].str.upper()


desc2=dft['PART NO_DESC'].values

X_N=vectorizer.transform(desc2)
predict2=clf2.predict(X_N)
predict2
dft['predict2']=predict2
dft.to_csv('c:\\temp\\predict.csv')


NameError: name 'dft' is not defined